<a href="https://colab.research.google.com/github/afvallejo/AD_2023/blob/main/03_pbmc_colab_annotation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Single-Cell Transcriptomics with Scanpy: A Google Colab Tutorial
### Description:
This tutorial walks you through the analysis of single-cell RNA sequencing data using Scanpy, specifically focusing on 3k peripheral blood mononuclear cells (PBMCs) as provided by 10X. You will learn how to load, preprocess, and visualize single-cell data in Python.

## Setup: Install Required Packages
Let's start by setting up our environment in Google Colab. We'll need to install `scanpy`, `matplotlib`, and a few other important packages.

In [ ]:
!pip install scanpy
!pip install anndata
!pip install matplotlib
!pip install seaborn
!pip install h5py
!pip install leidenalg

## Import Libraries
Now that the packages are installed, we need to import them.

In [ ]:
import scanpy as sc
import anndata as ad
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

## Load Data
The PBMC dataset can be downloaded directly from the Parse Biosciences website. Here, for educational purposes, we're going to use an example dataset that can be downloaded directly from Scanpy.

In [ ]:
# Download and load the PBMC dataset
adata = sc.datasets.pbmc3k()  # This is a demo PBMC dataset

# Display basic information about the dataset
adata

## Data Preprocessing
In this section, we'll preprocess the dataset, which includes filtering the cells, normalizing the data, and finding highly variable genes.

In [ ]:
# Filter out low-quality cells and genes
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)

# Calculate the percentage of mitochondrial genes in each cell
adata.var['mt'] = adata.var_names.str.startswith('MT-')
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

# Plot QC metrics to get an overview
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'], jitter=0.4, multi_panel=True)

# Filter cells based on quality control metrics
adata = adata[adata.obs.n_genes_by_counts < 2500, :]
adata = adata[adata.obs.pct_counts_mt < 5, :]

## Normalization & Log Transformation
Next, we'll normalize the data so that each cell has the same total read count. We'll then apply a log transformation to bring the data to a suitable scale for downstream analysis.

In [ ]:
# Normalize the data to 10,000 reads per cell
sc.pp.normalize_total(adata, target_sum=1e4)

# Log-transform the data
sc.pp.log1p(adata)

## Identify Highly Variable Genes
We now need to identify highly variable genes, as these will be informative for downstream analysis like clustering.

In [ ]:
# Ensure variable names are unique
adata.var_names_make_unique()

In [ ]:
# Identify highly variable genes
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)

# Plot the highly variable genes
sc.pl.highly_variable_genes(adata)

# Keep only highly variable genes
adata = adata[:, adata.var.highly_variable]

## Dimensionality Reduction
We use PCA to reduce the dimensionality of the dataset, which helps us to visualize and better understand our data.

In [ ]:
# Perform PCA for dimensionality reduction
sc.pp.scale(adata, max_value=10)
sc.tl.pca(adata, svd_solver='arpack')

# Visualize the PCA results
sc.pl.pca_variance_ratio(adata, log=True)

## Clustering
We can now proceed to cluster the cells based on gene expression patterns using neighborhood graphs and Louvain clustering.

In [ ]:
# Compute the neighborhood graph
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)

# Perform clustering using the Louvain algorithm
sc.tl.leiden(adata)

# Visualize the clustering using UMAP
sc.tl.umap(adata)
sc.pl.umap(adata, color=['leiden'])

## Marker Gene Analysis
We want to identify marker genes that define each cluster and use them to annotate cell types.

In [ ]:
# Find marker genes for each cluster
sc.tl.rank_genes_groups(adata, 'leiden', method='t-test')
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)

In [ ]:
#method : {‘logreg’, ‘t-test’, ‘wilcoxon’, ‘t-test_overestim_var’} | None (default: None)
import pandas as pd
sc.tl.rank_genes_groups(adata, 'leiden', method='wilcoxon',n_genes=adata.shape[1], rankby_abs=False,corr_method='bonferroni',key_added='wilcoxon_rank_genes_groups' )
wilcoxon=pd.DataFrame(adata.uns['wilcoxon_rank_genes_groups']['names']).head(20)
wilcoxon

## Student Exercise: Annotating Clusters

Now that we have identified clusters and marker genes, let's annotate the clusters.

### Question for Students:
- Based on the marker gene analysis, can you identify which cell types each cluster might represent?
- Look at the top marker genes for each cluster and use external resources (e.g., literature or online databases) to annotate the clusters.

### Problem:
- Write a code snippet to add your annotations to the clusters. Use the `adata.obs['louvain']` column to assign cell type labels.

## Example Solution:


In [ ]:
# Example of adding annotations
# Replace the cluster numbers with your identified cell types
cluster_annotations = {
    '0': 'Naive T cells',
    '1': 'Memory T cells',
    '2': 'B cells',
    '3': 'NK cells',
    '4': 'Monocytes'
}

# Annotate the clusters
adata.obs['cell_type'] = adata.obs['leiden'].map(cluster_annotations).astype('category')


In [ ]:
# Visualize the annotated clusters
sc.pl.umap(adata, color=['cell_type'])